<!--TITLE:Data Augmentation-->

# Introduction #

Now that you've learned the fundamentals of convolutional classifiers, you're ready to move on to more advanced topics.

In this lesson, you'll learn a trick that can give a boost to your image classifiers: it's called **data augmentation**. 

# The Usefulness of Fake Data #

The best way to improve the performance of a machine learning model is to train it on more data. The more examples the model has to learn from, the better it will be able to recognize which differences in images matter and which do not. More data helps the model to *generalize* better.

One easy way of getting more data is to use the data you already have. If we can transform the images in our dataset in ways that preserve the class, we can teach our classifier to ignore those kinds of transformations. For instance, whether a car is facing left or right in a photo doesn't change the fact that it is a *Car* and not a *Truck*. So, if we **augment** our training data with flipped images, our classifier will learn that "left or right" is a difference it should ignore.

And that's the whole idea behind data augmentation: add in some extra fake data that looks reasonably like the real data and your classifier will improve.

# Using Data Augmentation #

Typically, many kinds of transformation are used when augmenting a dataset. These might include rotating the image, adjusting the color or contrast, warping the image, or many other things, usually applied in combination. Here is a sample of the different ways a single image might be transformed.

<figure>
<img src="https://i.imgur.com/UaOm0ms.png" width=400, alt="Sixteen transformations of a single image of a car.">
</figure>

Data augmentation is usually done *online*, meaning, as the images are being fed into the network for training. Recall that training is usually done on mini-batches of data. This is what a batch of 16 images might look like when data augmentation is used.

<figure>
<img src="https://i.imgur.com/MFviYoE.png" width=400, alt="A batch of 16 images with various random transformations applied.">
</figure>

Each time an image is used during training, a new random transformation is applied. This way, the model is always seeing something a little different than what it's seen before. This extra variance in the training data is what helps the model on new data.

It's important to remember though that not every transformation will be useful on a given problem. Most importantly, whatever transformations you use should not mix up the classes. If you were training a [digit recognizer](https://www.kaggle.com/c/digit-recognizer), for instance, rotating images would mix up '9's and '6's. In the end, the best approach for finding good augmentations is the same as with most ML problems: try it and see!

# Example - Training with Data Augmentation #

Keras lets you augment your data in two ways. The first way is to include it in the data pipeline with a function like [`ImageDataGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator). The second way is to include it in the model definition by using Keras's **preprocessing layers**. This is the approach that we'll take. The primary advantage for us is that the image transformations will be computed on the GPU instead of the CPU, potentially speeding up training.

In this exercise, we'll learn how to improve the classifier from Lesson 1 through data augmentation. This next hidden cell sets up the data pipeline.

In [ ]:
#$HIDE_INPUT$
from cv_prelude import *

## Step 2 - Define Model ##

We'll continue with the VGG16 model we've used throughout this course. For the head, we've increased the number of hidden units from 6 to 8. Since there is now more variation in the data, we can afford a little extra capacity.

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
# these are a new feature in TF 2.2
import tensorflow.keras.layers.experimental.preprocessing as preprocessing


pretrained_base = tf.keras.models.load_model(
    '/kaggle/input/cv-course-models/cv-course-models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

model = keras.Sequential([
    # Preprocessing
    # randomly flip left to right
    preprocessing.RandomFlip(mode='horizontal'),
    # randomly rotate by as much as 20% either direction
    preprocessing.RandomRotation(factor=0.20),
    # randomly adjust the contrast by factors between 0.5 and 1.5
    preprocessing.RandomContrast(factor=0.5),

    # Base
    pretrained_base,

    # Head
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

## Step 3 - Train and Evaluate ##

And now we'll start the training.

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=20,
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();

Though data augmentation didn't seem to improve the accuracy of the classifier, it's interesting to notice that the loss curves stay much closer together. This might indicate that the augmentation had a *regularizing* effect on the network. The random transformations seem to have prevented the network from simply memorizing the training data.

# Your Turn #

Move on to the [**Exercise**](#$NEXT_NOTEBOOK_URL$) to apply data augmentation to the custom convnet you built in Lesson 5. This will be your best model ever!